In [ ]:
import os
import copy
import datasets as ds
import FPMLmodule.utils as utils
import tensorflow as tf
import FPMLmodule.backbones as backbones
import FPMLmodule.classifiers as classifiers
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from FPMLmodule.fpml import FPML 
from pathlib import Path

print('TensorFlow Version:', tf.__version__)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

weightsRN50 = "./weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsMNV2 = "./weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"
weightsENB2 = "./weights/efficientnetb2_notop.h5"
weightsINCEPTIONV3 = "./weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsXCEPTION = "./weights/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [ ]:
def researchBestModel(configureDS, datasets,epochsSearch, epochsBest, path, optimizers, learningRates, defaultHypers, losses, defaultBackbone, backbonesForSearch, verbose=1):
    trainDs, testDs, valDs = datasets
    imgDim = configureDS.inputDim
    nbClasses = len(configureDS.classNames)
    activation = "softmax"
    outFilePath = path + 'ablationHistory.csv'
    
    print('Ablation for', configureDS.name)
    # Transfer Learning Mode
    trainingModeStudy =  {
        'DNN-TransferLearningMode' : {
            "architecture": {
                "backbone": [
                    defaultBackbone(imgDim, weights=weightsMNV2, trainable=False, name='PT&Untrainable'),
                    defaultBackbone(imgDim, weights=weightsMNV2, trainable=True, name='PT&Trainable'),
                    defaultBackbone(imgDim, weights=None, trainable=True, name='IN&Trainable')
                    ],
                "classfier": classifiers.DefaultClassifier(nbClasses, activation),
                "inputLayer": None, 
                "inputDim": imgDim
            },
            "hyperparameters":defaultHypers
        }
    }
    trainingModeHistories = utils.researchStudies(trainDs, valDs, trainingModeStudy, epochsSearch, verbose)
    bestTrainMode = utils.getBestStudyFromHistories(trainingModeHistories)
    trainable = 'Trainable' in bestTrainMode
    pretrained = 'PT' in bestTrainMode
    utils.displayStudiesProgress(trainingModeHistories, path, 'Comparison Transfer Learning Mode')
    utils.saveStudyHistory(trainingModeStudy, trainingModeHistories, outFilePath)
    print("Backbone Transfer Learning Mode Selected - Pretrained:", pretrained, 'Trainable:', trainable)


    # loss function
    lossStudy = copy.deepcopy(trainingModeHistories[bestTrainMode]['config'])
    lossStudy['hyperparameters']["loss"] = losses
    lossesHistories = utils.researchStudies(trainDs, valDs, {bestTrainMode: lossStudy}, epochsSearch, verbose)
    lossesHistories[bestTrainMode+"_"+str(defaultHypers["loss"])] = trainingModeHistories[bestTrainMode]
    bestLoss = utils.getBestStudyFromHistories(lossesHistories)
    utils.displayStudiesProgress(lossesHistories, path, 'Comparison Loss Functions')
    utils.saveStudyHistory({'BestLoss': lossStudy}, lossesHistories, outFilePath)
    print("Best Loss Function:", str(lossesHistories[bestLoss]['config']['hyperparameters']["loss"]))
    
    # Learning rate
    learningRatesStudy = copy.deepcopy(lossesHistories[bestLoss]['config'])
    learningRatesStudy['hyperparameters']['learningRate'] = learningRates
    learningRatesHistories = utils.researchStudies(trainDs, valDs, {bestLoss: learningRatesStudy}, epochsSearch, verbose)
    learningRatesHistories[bestLoss+"_"+str(defaultHypers["learningRate"])] = lossesHistories[bestLoss]
    bestLearningRate = utils.getBestStudyFromHistories(learningRatesHistories)
    utils.displayStudiesProgress(learningRatesHistories, path, 'Comparison Learning Rate')
    utils.saveStudyHistory({'BestLearningRate': learningRatesStudy}, learningRatesHistories, outFilePath)
    print("Best Learning Rate:", str(learningRatesHistories[bestLearningRate]['config']["hyperparameters"]["learningRate"]))


    # Optimizer
    optimizersStudy = copy.deepcopy(learningRatesHistories[bestLearningRate]['config'])
    optimizersStudy["hyperparameters"]["optimizer"] = optimizers
    optimizersHistories = utils.researchStudies(trainDs, valDs, {bestLearningRate : optimizersStudy}, epochsSearch, verbose)
    optimizersHistories[bestLearningRate+"_"+defaultHypers['optimizer'].__name__] = learningRatesHistories[bestLearningRate]
    bestOptimizier = utils.getBestStudyFromHistories(optimizersHistories)
    utils.displayStudiesProgress(optimizersHistories, path, 'Comparison Optimizer')
    utils.saveStudyHistory({'BestOptimizier': optimizersStudy}, optimizersHistories, outFilePath)
    print("Best Optimizier:", optimizersHistories[bestOptimizier]['config']["hyperparameters"]["optimizer"].__name__)
    
    
    # DNN Backbone
    backboneConfigs = [backbone['backbone'](imgDim, weights=backbone['weights'], trainable=trainable) for backbone in backbonesForSearch]
    bestBackboneStudy = copy.deepcopy(optimizersHistories[bestOptimizier]['config'])
    bestBackboneStudy["architecture"]["backbone"] = backboneConfigs
    bestBackboneHistories = utils.researchStudies(trainDs, valDs, {bestOptimizier: bestBackboneStudy}, epochsSearch, verbose)
    bestBackboneHistories[bestOptimizier+"_"+defaultBackbone.__name__] = optimizersHistories[bestOptimizier]
    bestBackbone = utils.getBestStudyFromHistories(bestBackboneHistories)
    utils.displayStudiesProgress(bestBackboneHistories, path, 'Comparison DNN Backbone')
    utils.saveStudyHistory({'BestBackbone': bestBackboneStudy}, bestBackboneHistories, outFilePath)
    print("Best DNN Backbone:", bestBackboneHistories[bestBackbone]['config']["architecture"]["backbone"].name)

    bestModelName = bestBackbone
    bestModelConfiguration = bestBackboneHistories[bestBackbone]['config']
    trainValDs = trainDs.concatenate(valDs)

    bestModel = FPML(**bestModelConfiguration["architecture"]).create(**bestModelConfiguration["hyperparameters"])
    bestModelHistory = bestModel.fit(trainValDs, validation_data=testDs, epochs=epochsBest, verbose=verbose)
    toSave = {'Best Model':{'history' : bestModelHistory, 'config' : bestModelConfiguration}}
    utils.displayStudiesProgress(toSave, path, 'Best Model')
    utils.saveStudyHistory({'Best Model': bestModelConfiguration}, toSave, outFilePath)
    bestModel.save(path+configureDS.name+'_'+bestModelName+".h5")
    utils.displayConfusion(testDs, bestModel, path)


In [ ]:

# Dataset configuration
dsConfig = {
    'batchSize': 32, 
    'parallelTune': tf.data.AUTOTUNE, 
    'split': [0.7, 0.15, 0.15], 
    'inputDim': (120, 120, 3), 
    'seed': 9, 
    'shuffle': True
    }

configureDS = ds.SOCOFingGender(**dsConfig, sampling=ds.SOCOFingGender.UNDER_SAMPLING)
dbStudyOutPath = './out/'+configureDS.name+'/'
Path(dbStudyOutPath).mkdir(parents=True, exist_ok=True)

datasets = configureDS.create()
utils.datasetAnalysisAndDisplay(datasets, configureDS, dbStudyOutPath)

SOCOFingGenderConfig = {
    'path': dbStudyOutPath+'/',
    'configureDS': configureDS,
    'datasets': datasets,
    'defaultBackbone': backbones.MobileNetV2,
    'defaultHypers': {
        "optimizer": Adam,
        "learningRate": 0.001,
        "loss": 'binary_crossentropy',
        "metrics": 'accuracy'
    },
    'optimizers': [Nadam, RMSprop][:1],
    'learningRates': [0.1, 0.01, 0.0001, 0.00001][:1],
    'losses': ['binary_focal_crossentropy', 'hinge'][:1],
    'backbonesForSearch': [
        {'backbone' : backbones.ResNet50, 'weights' : weightsRN50},
        {'backbone' : backbones.EfficientNetB2, 'weights' : weightsENB2},
        {'backbone' : backbones.InceptionV3, 'weights' : weightsINCEPTIONV3},
        {'backbone' : backbones.Xception, 'weights' : weightsXCEPTION}, 
    ],
    'epochsSearch': 2,
    'epochsBest': 2,
    'verbose': 0
}

researchBestModel(**SOCOFingGenderConfig)